In [1]:
!pip install gradio==4.0.2 transformers torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.3/25.3 MB 24.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 5.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.7/302.7 kB 22.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 6.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1

In [3]:
import gradio as gr
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer
# AutoModelForSequenceClassification, AutoTokenizer => 튜닝하겠구나~

In [4]:
labels = [
    "여성/가족",
    "남성",
    "성소수자",
    "인종/국적",
    "연령",
    "지역",
    "종교",
    "기타 혐오",
    "악플/욕설",
    "clean",
]

모델 가져오기

In [6]:
model = AutoModelForSequenceClassification.from_pretrained('smilegate-ai/kor_unsmile')
tokenizer = AutoTokenizer.from_pretrained('smilegate-ai/kor_unsmile')

config.json:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/436M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/370 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/250k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/721k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

악플 감지 함수 구하기

In [7]:
labels

['여성/가족', '남성', '성소수자', '인종/국적', '연령', '지역', '종교', '기타 혐오', '악플/욕설', 'clean']

In [16]:
def checkHateSpeech(text):
  with torch.no_grad():
    # model의 gradient 계산 비활성화
    inputs = tokenizer(text, return_tensors='pt') # text를 받아서 pytorch 형태로 분절
    outputs = model(**inputs)
    # 변환된 입력(inputs) 모델에 넣어 결과 도출

    logits = outputs['logits']

    # 원시 예측값을 확률로 변환(sigmoid : 0~1 사이)
    probs = torch.sigmoid(logits)
    print(probs)

    # 0~1 사이 나오는 확률값 >> 라벨과 매칭 >> dict() 형태로 반환
    probs_by_labels = {labels[i] : float(probs[0][i]) for i in range(len(labels))}
    # probs[0][i] : probs 리스트의 첫 번째 요소에서 i번째
    return probs_by_labels

In [17]:
checkHateSpeech('이 중국 따거...짱개')

tensor([[0.0224, 0.0175, 0.0137, 0.9181, 0.0180, 0.0228, 0.0247, 0.0175, 0.0420,
         0.0361]])


{'여성/가족': 0.022381041198968887,
 '남성': 0.017535710707306862,
 '성소수자': 0.013692877255380154,
 '인종/국적': 0.9181445240974426,
 '연령': 0.01804455555975437,
 '지역': 0.022841311991214752,
 '종교': 0.024652032181620598,
 '기타 혐오': 0.017491037026047707,
 '악플/욕설': 0.04198802635073662,
 'clean': 0.036118995398283005}

 Gradio 화면 그리기

In [18]:
iface = gr.Interface(
    fn = checkHateSpeech,
    inputs = gr.Textbox(label='Text'),
    outputs = gr.Label(label='혐오 발언 분석', num_top_classes=5),
    examples = [
        '사랑해요~', '증오해요~'
    ]
)
iface.launch(share=True)

IMPORTANT: You are using gradio version 4.0.2, however version 4.29.0 is available, please upgrade.
--------
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://0b40cd4a1b00a94549.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
